# outputs

> Functions that emit markdown from the `outputs` element of notebook code cells

In [ ]:
#| default_exp outputs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
import io
from textwrap import dedent
from typing import Dict, Iterable

In [ ]:
# | exporti
def emit_lines(lines: Iterable[str], stream: io.TextIOBase):
    for line in lines:
        stream.write(line)
    if line[-1] != '\n':
        stream.write('\n')


def emit_plaintext(lines: Iterable[str], stream: io.TextIOBase):
    stream.write('```\n')
    emit_lines(lines, stream)
    stream.write('```\n')


def emit_output_data(data: Dict, stream: io.TextIOBase):
    lines_mimetypes = ['text/markdown', 'text/latex', 'text/html']

    for mimetype in lines_mimetypes:
        if mimetype in data:
            emit_lines(data[mimetype], stream)
            return

    if 'text/plain' in data:
        emit_plaintext(data['text/plain'], stream)

In [ ]:
# | export
def emit_stream_output(output: Dict, stream: io.TextIOBase):
    """Emits markdown from [stream output](https://nbformat.readthedocs.io/en/latest/format_description.html#stream-output)."""
    assert output['output_type'] == 'stream'

    text = output['text']
    emit_plaintext(text, stream)

In [ ]:
stream_test_cases = [
    {
        'name': 'Single line ending in a newline',
        'stream_output': {
            'name': 'stdout',
            'output_type': 'stream',
            'text': ['hello, world\n'],
        },
        'expected': dedent(
            """\
            ```
            hello, world
            ```
            """            
        )
    },
    {
        'name': 'Single line not ending in a newline',
        'stream_output': {
            'name': 'stdout',
            'output_type': 'stream',
            'text': ['hello, world'],
        },
        'expected': dedent(
            """\
            ```
            hello, world
            ```
            """            
        )
    },
    {
        'name': 'Multiline',
        'stream_output': {
            'name': 'stdout',
            'output_type': 'stream',
            'text': ['hello, world\n', 'a second line\n', 'a third line\n'],
        },
        'expected': dedent(
            """\
            ```
            hello, world
            a second line
            a third line
            ```
            """            
        )
    }
]

for case in stream_test_cases:
    name = case['name']
    stream_output = case['stream_output']
    expected = case['expected']

    stream = io.StringIO()

    emit_stream_output(stream_output, stream)

    stream.seek(0)
    output = stream.read()

    test_eq(output, expected)
    print(f"Case: {name}")
    print(output)


Case: Single line ending in a newline
```
hello, world
```

Case: Single line not ending in a newline
```
hello, world
```

Case: Multiline
```
hello, world
a second line
a third line
```



In [ ]:
# | export
def emit_execute_result_output(output: Dict, stream: io.TextIOBase):
    """Emits markdown from [`execute_result`](https://nbformat.readthedocs.io/en/latest/format_description.html#execute-result) outputs"""

    assert output['output_type'] == 'execute_result'
    data = output['data']
    emit_output_data(data, stream)

In [ ]:
execution_result_test_cases = [
    {
        'name': 'Markdown',
        'execute_result_output': {
            'output_type': 'execute_result',
            'data': {
                'text/markdown': [
                    '$$\\begin{bmatrix}\n',
                    '{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\\n',
                    '{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\\n',
                    '{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\\n',
                    '{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}\n',
                    '$$',
                ],
                'text/plain': ['<IPython.core.display.Markdown object>'],
            },
        },
        'expected': dedent(
            """\
            $$\\begin{bmatrix}
            {x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\\\
            {x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\\\
            {x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\\\
            {x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\\end{bmatrix}
            $$
            """
        ),
    },
    {
        'name': 'Latex',
        'execute_result_output': {
            'output_type': 'execute_result',
            'data': {
                'text/latex': [
                    '$\\displaystyle b^{[2]} + {W^{[2]}}_{1,1} {a^{[1]}}_{1,1} + {W^{[2]}}_{1,2} {a^{[1]}}_{1,2} + {W^{[2]}}_{1,3} {a^{[1]}}_{1,3} + {W^{[2]}}_{2,1} {a^{[1]}}_{2,1} + {W^{[2]}}_{2,2} {a^{[1]}}_{2,2} + {W^{[2]}}_{2,3} {a^{[1]}}_{2,3} + {W^{[2]}}_{3,1} {a^{[1]}}_{3,1} + {W^{[2]}}_{3,2} {a^{[1]}}_{3,2} + {W^{[2]}}_{3,3} {a^{[1]}}_{3,3}$'
                ],
                'text/plain': [
                    'b^{[2]} + W^{[2]}[1, 1]*a^{[1]}[1, 1] + W^{[2]}[1, 2]*a^{[1]}[1, 2] + W^{[2]}[1, 3]*a^{[1]}[1, 3] + W^{[2]}[2, 1]*a^{[1]}[2, 1] + W^{[2]}[2, 2]*a^{[1]}[2, 2] + W^{[2]}[2, 3]*a^{[1]}[2, 3] + W^{[2]}[3, 1]*a^{[1]}[3, 1] + W^{[2]}[3, 2]*a^{[1]}[3, 2] + W^{[2]}[3, 3]*a^{[1]}[3, 3]'
                ],
            },
        },
        'expected': dedent(
            """\
            $\\displaystyle b^{[2]} + {W^{[2]}}_{1,1} {a^{[1]}}_{1,1} + {W^{[2]}}_{1,2} {a^{[1]}}_{1,2} + {W^{[2]}}_{1,3} {a^{[1]}}_{1,3} + {W^{[2]}}_{2,1} {a^{[1]}}_{2,1} + {W^{[2]}}_{2,2} {a^{[1]}}_{2,2} + {W^{[2]}}_{2,3} {a^{[1]}}_{2,3} + {W^{[2]}}_{3,1} {a^{[1]}}_{3,1} + {W^{[2]}}_{3,2} {a^{[1]}}_{3,2} + {W^{[2]}}_{3,3} {a^{[1]}}_{3,3}$
            """
        ),
    },
    {
        'name': 'HTML',
        'execute_result_output': {
            'output_type': 'execute_result',
            'data': {
                'text/html': [
                    '\n',
                    '        <iframe\n',
                    '            width=\"400\"\n',
                    '            height=\"300\"\n',
                    '            src=\"https://www.youtube.com/embed/Y7eoO3hZQYY?loop=1&mute=1&playlist=Y7eoO3hZQYY&autoplay=1\"\n',
                    '            frameborder=\"0\"\n',
                    '            allowfullscreen\n',
                    '            allow=\"autoplay\"\n',
                    '        ></iframe>\n',
                    '        ',
                ],
                'text/plain': ['<IPython.lib.display.YouTubeVideo>'],
            },
        },
        'expected': (
            '\n'
            '        <iframe\n'
            '            width=\"400\"\n'
            '            height=\"300\"\n'
            '            src=\"https://www.youtube.com/embed/Y7eoO3hZQYY?loop=1&mute=1&playlist=Y7eoO3hZQYY&autoplay=1\"\n'
            '            frameborder=\"0\"\n'
            '            allowfullscreen\n'
            '            allow=\"autoplay\"\n'
            '        ></iframe>\n'
            '        \n'
        ),
    },
    {
        'name': 'Plaintext',
        'execute_result_output': {
            'output_type': 'execute_result',
            'data': {
                'text/plain': [
                    'array([[w_{11}*x_{11} + w_{12}*x_{12} + w_{21}*x_{21} + w_{22}*x_{22},\n',
                    '        w_{11}*x_{12} + w_{12}*x_{13} + w_{21}*x_{22} + w_{22}*x_{23},\n',
                    '        w_{11}*x_{13} + w_{12}*x_{14} + w_{21}*x_{23} + w_{22}*x_{24}],\n',
                    '       [w_{11}*x_{21} + w_{12}*x_{22} + w_{21}*x_{31} + w_{22}*x_{32},\n',
                    '        w_{11}*x_{22} + w_{12}*x_{23} + w_{21}*x_{32} + w_{22}*x_{33},\n',
                    '        w_{11}*x_{23} + w_{12}*x_{24} + w_{21}*x_{33} + w_{22}*x_{34}],\n',
                    '       [w_{11}*x_{31} + w_{12}*x_{32} + w_{21}*x_{41} + w_{22}*x_{42},\n',
                    '        w_{11}*x_{32} + w_{12}*x_{33} + w_{21}*x_{42} + w_{22}*x_{43},\n',
                    '        w_{11}*x_{33} + w_{12}*x_{34} + w_{21}*x_{43} + w_{22}*x_{44}]],\n',
                    '      dtype=object)',
                ]
            },
        },
        'expected': dedent(
            """\
            ```
            array([[w_{11}*x_{11} + w_{12}*x_{12} + w_{21}*x_{21} + w_{22}*x_{22},
                    w_{11}*x_{12} + w_{12}*x_{13} + w_{21}*x_{22} + w_{22}*x_{23},
                    w_{11}*x_{13} + w_{12}*x_{14} + w_{21}*x_{23} + w_{22}*x_{24}],
                   [w_{11}*x_{21} + w_{12}*x_{22} + w_{21}*x_{31} + w_{22}*x_{32},
                    w_{11}*x_{22} + w_{12}*x_{23} + w_{21}*x_{32} + w_{22}*x_{33},
                    w_{11}*x_{23} + w_{12}*x_{24} + w_{21}*x_{33} + w_{22}*x_{34}],
                   [w_{11}*x_{31} + w_{12}*x_{32} + w_{21}*x_{41} + w_{22}*x_{42},
                    w_{11}*x_{32} + w_{12}*x_{33} + w_{21}*x_{42} + w_{22}*x_{43},
                    w_{11}*x_{33} + w_{12}*x_{34} + w_{21}*x_{43} + w_{22}*x_{44}]],
                  dtype=object)
            ```
            """
        ),
    },
]


for case in execution_result_test_cases:
    name = case['name']
    execute_result_output = case['execute_result_output']
    expected = case['expected']

    stream = io.StringIO()

    emit_execute_result_output(execute_result_output, stream)

    stream.seek(0)
    output = stream.read()

    test_eq(output, expected)
    print(f"Case: {name}")
    print(output)

Case: Markdown
$$\begin{bmatrix}
{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\
{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\
{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\
{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\end{bmatrix}
$$

Case: Latex
$\displaystyle b^{[2]} + {W^{[2]}}_{1,1} {a^{[1]}}_{1,1} + {W^{[2]}}_{1,2} {a^{[1]}}_{1,2} + {W^{[2]}}_{1,3} {a^{[1]}}_{1,3} + {W^{[2]}}_{2,1} {a^{[1]}}_{2,1} + {W^{[2]}}_{2,2} {a^{[1]}}_{2,2} + {W^{[2]}}_{2,3} {a^{[1]}}_{2,3} + {W^{[2]}}_{3,1} {a^{[1]}}_{3,1} + {W^{[2]}}_{3,2} {a^{[1]}}_{3,2} + {W^{[2]}}_{3,3} {a^{[1]}}_{3,3}$

Case: HTML

        <iframe
            width="400"
            height="300"
            src="https://www.youtube.com/embed/Y7eoO3hZQYY?loop=1&mute=1&playlist=Y7eoO3hZQYY&autoplay=1"
            frameborder="0"
            allowfullscreen
            allow="autoplay"
        ></iframe>
        

Case: Plaintext
```
array([[w_{11}*x_{11} + w_{12}*x_{12} + w_{21}*x_{21} + w_{22}*x_{22},
        w_{11}*x_{12} + w_{12

In [ ]:
# | export
def emit_display_data_output(output: Dict, stream: io.TextIOBase):
    """Emits markdown from [`display_data`](https://nbformat.readthedocs.io/en/latest/format_description.html#display-data) outputs"""

    assert output['output_type'] == 'display_data'
    data = output['data']
    emit_output_data(data, stream)

In [ ]:
display_data_test_cases = [
    {
        'name': tc['name'],
        'display_data_output': {
            'output_type': 'display_data',
            'data': tc['execute_result_output']['data']
        },
        'expected': tc['expected']
    } for tc in execution_result_test_cases
]

for case in display_data_test_cases:
    name = case['name']
    display_data_output = case['display_data_output']
    expected = case['expected']

    stream = io.StringIO()

    emit_display_data_output(display_data_output, stream)

    stream.seek(0)
    output = stream.read()

    test_eq(output, expected)
    print(f"Case: {name}")
    print(output)
    

Case: Markdown
$$\begin{bmatrix}
{x}_{1,1} & {x}_{1,2} & {x}_{1,3} & {x}_{1,4}\\
{x}_{2,1} & {x}_{2,2} & {x}_{2,3} & {x}_{2,4}\\
{x}_{3,1} & {x}_{3,2} & {x}_{3,3} & {x}_{3,4}\\
{x}_{4,1} & {x}_{4,2} & {x}_{4,3} & {x}_{4,4}\end{bmatrix}
$$

Case: Latex
$\displaystyle b^{[2]} + {W^{[2]}}_{1,1} {a^{[1]}}_{1,1} + {W^{[2]}}_{1,2} {a^{[1]}}_{1,2} + {W^{[2]}}_{1,3} {a^{[1]}}_{1,3} + {W^{[2]}}_{2,1} {a^{[1]}}_{2,1} + {W^{[2]}}_{2,2} {a^{[1]}}_{2,2} + {W^{[2]}}_{2,3} {a^{[1]}}_{2,3} + {W^{[2]}}_{3,1} {a^{[1]}}_{3,1} + {W^{[2]}}_{3,2} {a^{[1]}}_{3,2} + {W^{[2]}}_{3,3} {a^{[1]}}_{3,3}$

Case: HTML

        <iframe
            width="400"
            height="300"
            src="https://www.youtube.com/embed/Y7eoO3hZQYY?loop=1&mute=1&playlist=Y7eoO3hZQYY&autoplay=1"
            frameborder="0"
            allowfullscreen
            allow="autoplay"
        ></iframe>
        

Case: Plaintext
```
array([[w_{11}*x_{11} + w_{12}*x_{12} + w_{21}*x_{21} + w_{22}*x_{22},
        w_{11}*x_{12} + w_{12

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()